In [1]:
import math
import sys,os
import numpy as np
import pandas as pd

sys.path.append('/Users/Ziqi/Downloads/mgwr/')
from gwr.gwr import GWR,GWRResults
from gwr.sel_bw import Sel_BW
from spglm.family import Gaussian, Poisson, Binomial

In [2]:
zillow = pd.read_csv("Zillow-test-dataset/zillow_5k.csv",sep=',')
zillow.head()

,utmX,utmY,value,nbaths,nbeds,area,age
0,379979.957285,3.752562e+06,87.397,2.0,3.0,1311.0,61.0
1,411210.623578,3.762339e+06,121.211,2.0,4.0,1200.0,62.0
2,400287.486356,3.743925e+06,76.145,1.0,2.0,724.0,70.0
3,391029.680899,3.776406e+06,136.320,2.0,3.0,1890.0,54.0
4,371824.111160,3.750099e+06,100.499,4.0,5.0,2166.0,71.0


In [3]:
#Converting things into matrices
y = zillow.value.as_matrix().reshape(-1,1)
X = zillow.iloc[:,3:].as_matrix()
k = zillow.shape[1]
u = zillow.utmX
v = zillow.utmY
n = zillow.shape[0]
coords = list(zip(u,v))

In [4]:
%%time
bw = 500
pysal_Result=GWR(coords,y,X,bw).fit()

CPU times: user 26.4 s, sys: 1.72 s, total: 28.1 s
Wall time: 16.7 s


In [ ]:
#Running FastGWR 1 Processor
!mpiexec -np 1 python fastgwr-mpi.py -data Zillow-test-dataset/zillow_5k.csv -out results.csv -a -c -bw 500

Starting FastGWR with 1 Processors
Data Input Path: Zillow-test-dataset/zillow_5k.csv
Output Result Path: results.csv
Constant: True
Spatial Kernel: Adaptive Bisquare
Fitting GWR Using Bandwidth: 500
Diagnostic Information:
AICc: 62399.20315246709
ENP: 132.5582097758877
R2: 0.6986624931885277
Total Time Elapsed: 3.09 seconds


In [ ]:
#Running FastGWR 2 Processor
!mpiexec -np 2 python fastgwr-mpi.py -data Zillow-test-dataset/zillow_5k.csv -out results.csv -a -c -bw 500

In [ ]:
print("pysal AICc:", pysal_Result.aicc)
print("pysal R2:",pysal_Result.R2)

In [ ]:
fastGWR_result = pd.read_csv("results.csv")
fastGWR_result.head()

In [ ]:
#Validate residual
np.allclose(fastGWR_result.residual,pysal_Result.resid_response.reshape(-1),atol=1e-4)

In [ ]:
#Validate Params
np.allclose(np.array(fastGWR_result.iloc[:,3:8]),pysal_Result.params,atol=1e-3)

In [ ]:
#Validate SEs
np.allclose(np.array(fastGWR_result.iloc[:,8:13]),pysal_Result.bse,rtol=1e-2)